In [1]:
# grab entire notebook name without the extension
DATE_TASK = "2022-05-15-NCBI-taxonomy-in-neo4j"
DIR = mkpath("$(homedir())/workspace/$DATE_TASK")
cd(DIR)
DATE, TASK = match(r"^(\d{4}-\d{2}-\d{2})-(.*)$", DATE_TASK).captures

2-element Vector{Union{Nothing, SubString{String}}}:
 "2022-05-15"
 "NCBI-taxonomy-in-neo4j"

In [2]:
# NEO4J_BIN_DIR = "/home/jupyter-cjprybol/software/neo4j-community-4.4.3/bin"
# if !occursin(NEO4J_BIN_DIR, ENV["PATH"])
#     ENV["PATH"] = "$(NEO4J_BIN_DIR):" * ENV["PATH"]
# end

In [3]:
database_id = "41a45c7d"
# DOMAIN = "41a45c7d.databases.neo4j.io"
NODES_FILE = "$(DIR)/ncbi_taxonomy.nodes.tsv"
EDGES_FILE = "$(DIR)/ncbi_taxonomy.edges.tsv"
USERNAME="neo4j"
# PASSWORD=readline(joinpath(homedir(), ".config", "neo4j", "ncbi-taxonomy.password.txt"));
PASSWORD = "PtvnGnnS58elLLVsGwfeovOLi5xdfPe0hTMrwa_WtbM"
ADDRESS="neo4j+s://$(database_id).databases.neo4j.io:7687"
# ADDRESS="neo4j+s://$(DOMAIN):7687"
NEO4J_IMPORT_DIRECTORY="/var/lib/neo4j/import"
DATABASE = "neo4j"

"neo4j"

In [4]:
function cypher(;address, username, password, database, cmd)
    return `cypher-shell --address $address --username $username --password $password --database $(database) --format auto $(cmd)`
end

cypher (generic function with 1 method)

In [5]:
import Pkg
pkgs = [
    "DataFrames",
    "ProgressMeter",
    "Graphs",
    "MetaGraphs",
    "uCSV"
]

for pkg in pkgs
    try
        Pkg.add(pkg)
    catch
#         # tried to install an unregistered local package
    end
    eval(Meta.parse("import $pkg"))
end

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`
   Resolving package versions...
  No Changes to `~/git/Mycelia/docs/Project.toml`
  No Changes to `~/git/Mycelia/docs/Manifest.toml`


In [6]:
# sudo apt-get remove neo4j
# sudo rm -rf /var/lib/neo4j
# sudo rm -rf /etc/neo4j
# sudo rm -rf /var/log/neo4j
# sudo rm -rf /var/lib/neo4j
# sudo apt-get install neo4j

In [17]:
function list_databases(;address="neo4j://localhost:7687", username="neo4j", password="password")
    @show password
    cmd = `cypher-shell --address $(address) --username $(username) --password $(password) --database system --format auto "show databases"`
    @show cmd
    io = open(cmd)
    return DataFrames.DataFrame(uCSV.read(io, header=1, quotes='"', encodings=Dict("FALSE" => false, "TRUE" => true))...)
end

list_databases (generic function with 1 method)

In [18]:
# # run(`sudo neo4j stop`)
# # run(`sudo neo4j status`)


In [31]:
run(`sudo neo4j start`)

Directories in use:
home:         /var/lib/neo4j
config:       /etc/neo4j
logs:         /var/lib/neo4j/logs
plugins:      /var/lib/neo4j/plugins
import:       /var/lib/neo4j
data:         /var/lib/neo4j/data
certificates: /var/lib/neo4j/certificates
licenses:     /var/lib/neo4j/licenses
run:          /var/lib/neo4j/run
Starting Neo4j.
Started neo4j (pid:30359). It is available at http://localhost:7474
There may be a short delay until the server is ready.


Process(`sudo neo4j start`, ProcessExited(0))

In [35]:
run(`neo4j status`)

Neo4j is running at pid 30668


Process(`neo4j status`, ProcessExited(0))

```
Permission denied.

The credentials you provided were valid, but must be changed before you can use this instance. If this is the first time you are using Neo4j, this is to ensure you are not using the default credentials in production. If you are not using default credentials, you are getting this message because an administrator requires a password change
```

In [39]:
# run(`sudo neo4j-admin set-initial-password testpass`)

Selecting JVM - Version:11.0.15+10-Ubuntu-0ubuntu0.18.04.1, Name:OpenJDK 64-Bit Server VM, Vendor:Private Build


Changed password for user 'neo4j'.


Process(`sudo neo4j-admin set-initial-password testpass`, ProcessExited(0))

In [42]:
# had to stop and restart to get the password change to take effect
# run(`sudo neo4j stop`)
# run(`sudo neo4j start`)

In [77]:
# p = "neo4j"
local_password = "testpass"
# p = "7^9\$qMv"
list_databases(password=local_password)

password = "testpass"
cmd = `cypher-shell --address neo4j://localhost:7687 --username neo4j --password testpass --database system --format auto 'show databases'`


,name,aliases,access,address,role,requestedStatus,currentStatus
,String,String,String,String,String,String,String
1,neo4j,[],read-write,localhost:7687,standalone,online,online
2,system,[],read-write,localhost:7687,standalone,online,online


In [49]:
cmd="MATCH (n) RETURN n LIMIT 10"
run(cypher(address=local_address, username=USERNAME, password=local_password, database=DATABASE, cmd=cmd))

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password testpass --database neo4j --format auto 'MATCH (n) RETURN n LIMIT 10'`, ProcessExited(0))

In [44]:
local_address="neo4j://localhost:7687"

"neo4j://localhost:7687"

In [48]:
cmd="MATCH (n) DETACH DELETE n"
run(cypher(address=local_address, username=USERNAME, password=local_password, database=DATABASE, cmd=cmd))

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password testpass --database neo4j --format auto 'MATCH (n) DETACH DELETE n'`, ProcessExited(0))

In [50]:
cmd="MATCH (n) RETURN n LIMIT 10"
run(cypher(address=local_address, username=USERNAME, password=local_password, database=DATABASE, cmd=cmd))

Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password testpass --database neo4j --format auto 'MATCH (n) RETURN n LIMIT 10'`, ProcessExited(0))

In [51]:
# works but can't update locally, need to push and restart kernel to activate changes
# "https://github.com/cjprybol/Mycelia.git#master",
# didn't work
# "$(homedir())/git/Mycelia#master",
pkg_path = "$(homedir())/git/Mycelia"
try
    eval(Meta.parse("import $(basename(pkg_path))"))
catch
    # Pkg.add(url=pkg)
    Pkg.develop(path=pkg_path)
    # pkg = replace(basename(pkg), ".git#master" => "")
    # pkg = replace(basename(pkg), "#master" => "")
    eval(Meta.parse("import $(basename(pkg_path))"))
end

## obtain NCBI taxonomy

In [52]:
taxdump_url = "https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz"
taxdump_local_tarball = "$(DIR)/$(basename(taxdump_url))"

if !isfile(taxdump_local_tarball)
    download(taxdump_url, taxdump_local_tarball)
end

taxdump_out = replace(taxdump_local_tarball, ".tar.gz" => "")
if !isdir(taxdump_out)
    mkpath(taxdump_out)
    run(`tar -xvzf $(taxdump_local_tarball) -C $(taxdump_out)`)
end

readdir(taxdump_out)

9-element Vector{String}:
 "citations.dmp"
 "delnodes.dmp"
 "division.dmp"
 "gc.prt"
 "gencode.dmp"
 "merged.dmp"
 "names.dmp"
 "nodes.dmp"
 "readme.txt"

In [53]:
#     Here we will create an in-memory dataframe to capture the contents of the names.dmp file

# Taxonomy names file (names.dmp):
# 	tax_id					-- the id of node associated with this name
# 	name_txt				-- name itself
# 	unique name				-- the unique variant of this name if name not unique
# 	name class				-- (synonym, common name, ...)

names_dmp = DataFrames.DataFrame(
    tax_id = String[],
    name_txt = String[],
    unique_name = String[],
    name_class = String[]
)
ProgressMeter.@showprogress for line in split(read(open("$(taxdump_out)/names.dmp"), String), "\t|\n")
    if isempty(line)
        continue
    else
        (tax_id, name_txt, unique_name, name_class) = split(line, "\t|\t")
        # tax_id = parse(Int, tax_id_string)
        row = (;tax_id, name_txt, unique_name, name_class)
        push!(names_dmp, row)
    end
end
names_dmp
# all are unique
# unique!(names_dmp)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:2339m


,tax_id,name_txt
,String,String
1,1,all
2,1,root
3,2,Bacteria
4,2,bacteria
5,2,eubacteria
6,2,Monera
7,2,Procaryotae
8,2,Prokaryotae
9,2,Prokaryota


In [54]:
names_dmp_transformed = DataFrames.DataFrame(
    Dict(
    [nc => Union{Missing, String}[] for nc in unique(names_dmp[!, "name_class"])]...,
    "tax_id" => String[]
    )
)

ProgressMeter.@showprogress for identifier_group in DataFrames.groupby(names_dmp, "tax_id")
    fields = Dict{String, Union{String, Missing}}(n => missing for n in names(names_dmp_transformed))
    for name_class in DataFrames.groupby(identifier_group, "name_class")
        nc = name_class[1, "name_class"]
        all_names = unique(filter(n -> !isempty(n), (name_class[!, "name_txt"]...,name_class[!, "unique_name"]...)))
        fields[nc] = join(all_names, ";;;")
    end
    fields["tax_id"] = identifier_group[1, "tax_id"]
    # @show fields
    push!(names_dmp_transformed, fields)
end
names_dmp_transformed

Progress: 100%|█████████████████████████████████████████| Time: 0:06:00m31m


,acronym,authority
,String?,String?
1,missing,missing
2,missing,missing
3,missing,Azorhizobium Dreyfus et al. 1988 emend. Lang et al. 2013
4,missing,Azorhizobium caulinodans Dreyfus et al. 1988
5,missing,Buchnera aphidicola Munson et al. 1991
6,missing,"Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Humphry et al. 2003;;;Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Suarez et al. 2014;;;""Cellvibrio"" Winogradsky 1929"
7,missing,"""Cellulomonas gilvus"" (Hulcher and King 1958) Christopherson et al. 2013;;;""Cellvibrio gilvus"" Hulcher and King 1958"
8,missing,Dictyoglomus Saiki et al. 1985
9,missing,Dictyoglomus thermophilum Saiki et al. 1985


In [55]:
#     Here we can see that there are divisions projected onto the tree that will allow easy grouping by taxonomic "group"s such as primates, viruses, phages, etc.

division_table = DataFrames.DataFrame(
    division_id = String[],
    division_name = String[],
    division_cde = String[],
    notes = String[]
)
for line in split(read(open("$(taxdump_out)/division.dmp"), String), "\t|\n")
    if !isempty(line)
        (division_id, division_cde, division_name, notes) = split(line, "\t|\t")
        row = (;division_id, division_cde, division_name, notes)
        push!(division_table, row)
    end
end
division_table

,division_id,division_name,division_cde,notes
,String,String,String,String
1,0,Bacteria,BCT,
2,1,Invertebrates,INV,
3,2,Mammals,MAM,
4,3,Phages,PHG,
5,4,Plants and Fungi,PLN,
6,5,Primates,PRI,
7,6,Rodents,ROD,
8,7,Synthetic and Chimeric,SYN,
9,8,Unassigned,UNA,No species nodes should inherit this division assignment


In [56]:
node_table = DataFrames.DataFrame(
    tax_id = String[],
    parent_tax_id = String[],
    rank = String[],
    embl_code = String[],
    division_id = String[]
)
ProgressMeter.@showprogress for line in split(read(open("$(taxdump_out)/nodes.dmp"), String), "\t|\n")
    if isempty(line)
        continue
    else
        (tax_id, parent_tax_id, rank, embl_code, division_id) = split(line, "\t|\t")
        row = (;tax_id, parent_tax_id, rank, embl_code, division_id)
        push!(node_table, row)
    end
end
node_table

Progress: 100%|█████████████████████████████████████████| Time: 0:00:286:45


,tax_id,parent_tax_id,rank,embl_code,division_id
,String,String,String,String,String
1,1,1,no rank,,8
2,2,131567,superkingdom,,0
3,6,335928,genus,,0
4,7,6,species,AC,0
5,9,32199,species,BA,0
6,10,1706371,genus,,0
7,11,1707,species,CG,0
8,13,203488,genus,,0
9,14,13,species,DT,0


In [57]:
joint_node_metadata = 
DataFrames.outerjoin(
    names_dmp_transformed, 
    DataFrames.outerjoin(node_table, division_table, on="division_id"),
    on="tax_id"
)

,acronym,authority
,String?,String?
1,missing,missing
2,missing,missing
3,missing,Azorhizobium Dreyfus et al. 1988 emend. Lang et al. 2013
4,missing,Azorhizobium caulinodans Dreyfus et al. 1988
5,missing,Buchnera aphidicola Munson et al. 1991
6,missing,"Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Humphry et al. 2003;;;Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Suarez et al. 2014;;;""Cellvibrio"" Winogradsky 1929"
7,missing,"""Cellulomonas gilvus"" (Hulcher and King 1958) Christopherson et al. 2013;;;""Cellvibrio gilvus"" Hulcher and King 1958"
8,missing,Dictyoglomus Saiki et al. 1985
9,missing,Dictyoglomus thermophilum Saiki et al. 1985


In [58]:
# joint_node_metadata[!, "identifier"] .= map(id -> "tax_id_$(id)", joint_node_metadata[!, "tax_id"])
joint_node_metadata[!, "identifier"] .= joint_node_metadata[!, "tax_id"]

2420860-element Vector{String}:
 "1"
 "2"
 "6"
 "7"
 "9"
 "10"
 "11"
 "13"
 "14"
 "16"
 "17"
 "18"
 "19"
 ⋮
 "2943662"
 "2943663"
 "2943664"
 "2943665"
 "2943705"
 "2943798"
 "2943938"
 "2943944"
 "2943985"
 "2943987"
 "2943991"
 "2943994"

In [59]:
joint_node_metadata[!, "TYPE"] .= "Taxonomy"

2420860-element Vector{String}:
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 ⋮
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"
 "Taxonomy"

In [60]:
joint_node_metadata = joint_node_metadata[!, DataFrames.Not("acronym")]

,authority
,String?
1,missing
2,missing
3,Azorhizobium Dreyfus et al. 1988 emend. Lang et al. 2013
4,Azorhizobium caulinodans Dreyfus et al. 1988
5,Buchnera aphidicola Munson et al. 1991
6,"Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Humphry et al. 2003;;;Cellvibrio (ex Winogradsky 1929) Blackall et al. 1986 emend. Suarez et al. 2014;;;""Cellvibrio"" Winogradsky 1929"
7,"""Cellulomonas gilvus"" (Hulcher and King 1958) Christopherson et al. 2013;;;""Cellvibrio gilvus"" Hulcher and King 1958"
8,Dictyoglomus Saiki et al. 1985
9,Dictyoglomus thermophilum Saiki et al. 1985


In [61]:
node_table = joint_node_metadata
for re in (r"\s+", r"-")
    for col in filter(n -> occursin(re, n), names(node_table))
        rename_map = col => replace(col, re => "_")
        @show rename_map
        DataFrames.rename!(node_table, rename_map)
    end
end

rename_map = "blast name" => "blast_name"
rename_map = "common name" => "common_name"
rename_map = "equivalent name" => "equivalent_name"
rename_map = "genbank acronym" => "genbank_acronym"
rename_map = "genbank common name" => "genbank_common_name"
rename_map = "scientific name" => "scientific_name"
rename_map = "type material" => "type_material"
rename_map = "in-part" => "in_part"


In [62]:
ProgressMeter.@showprogress for col in names(node_table)
    # strip away single-quotes
    node_table[!, col] .= map(x -> x isa AbstractString ? replace(x, "'" => "") : x, node_table[!, col])
    # and double quotes
    node_table[!, col] .= map(x -> x isa AbstractString ? replace(x, '"' => "") : x, node_table[!, col])
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


In [63]:
edge_table = DataFrames.rename(joint_node_metadata[!, ["parent_tax_id", "identifier"]], ["identifier" => "src", "parent_tax_id" => "dst"])
edge_table[!, "TYPE"] .= "IS_PARENT_OF"
edge_table

,dst,src,TYPE
,String,String,String
1,1,1,IS_PARENT_OF
2,131567,2,IS_PARENT_OF
3,335928,6,IS_PARENT_OF
4,6,7,IS_PARENT_OF
5,32199,9,IS_PARENT_OF
6,1706371,10,IS_PARENT_OF
7,1707,11,IS_PARENT_OF
8,203488,13,IS_PARENT_OF
9,13,14,IS_PARENT_OF


In [64]:
local_address="neo4j://localhost:7687"
# local_password="7^9\$qMv"
# local_password="neo4j"

"neo4j"

In [70]:
neo4j_import = "/var/lib/neo4j"
run(`sudo chmod 777 $(neo4j_import)`)
temp_upload_dir = mkpath(neo4j_import)

"/var/lib/neo4j"

In [71]:
NODE_TYPE = node_table[1, "TYPE"]

"Taxonomy"

In [72]:
parameters = ["$(n): row.$(n)" for n in filter(x -> x != "TYPE", names(node_table))]
parameters = "{" * join(parameters, ", ") * "}"

"{authority: row.authority, blast_name: row.blast_name, common_name: row.common_name, equivalent_name: row.equivalent_name, genbank_acronym: row.genbank_acronym, genbank_common_name: row.genbank_common_name, in_part: row.in_part, includes: row.includes, scientific_name: row.scientific_name, synonym: row.synonym, tax_id: row.tax_id, type_material: row.type_material, parent_tax_id: row.parent_tax_id, rank: row.rank, embl_code: row.embl_code, division_id: row.division_id, division_name: row.division_name, division_cde: row.division_cde, notes: row.notes, identifier: row.identifier}"

In [73]:
# THIS IS SO IMPORTANT FOR MAKING THINGS GO FAST
# AUTO SETS AN INDEX ON THESE PROPERTIES

In [79]:
local_password

"testpass"

In [80]:
cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.tax_id IS UNIQUE"
@time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.`scientific name` IS UNIQUE"
@time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.identifier IS UNIQUE"
@time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

  1.339495 seconds (135 allocations: 7.453 KiB)
  1.175489 seconds (238 allocations: 16.469 KiB)
  1.174746 seconds (238 allocations: 16.469 KiB)


Process(`cypher-shell --address neo4j://localhost:7687 --username neo4j --password testpass --database neo4j --format auto 'CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.identifier IS UNIQUE'`, ProcessExited(0))

In [81]:
window_size = 10000
V = DataFrames.nrow(node_table)
windows = [i:min(i+window_size-1,V) for i in 1:window_size:V]

243-element Vector{UnitRange{Int64}}:
 1:10000
 10001:20000
 20001:30000
 30001:40000
 40001:50000
 50001:60000
 60001:70000
 70001:80000
 80001:90000
 90001:100000
 100001:110000
 110001:120000
 120001:130000
 ⋮
 2310001:2320000
 2320001:2330000
 2330001:2340000
 2340001:2350000
 2350001:2360000
 2360001:2370000
 2370001:2380000
 2380001:2390000
 2390001:2400000
 2400001:2410000
 2410001:2420000
 2420001:2420860

In [82]:
# add the nodes to it

In [85]:
ProgressMeter.@showprogress for (i, w) in enumerate(windows)
    df_sub = node_table[w, :]
    f = "node$i.tsv"
    local_f_path = "$(temp_upload_dir)/$(f)"
    uCSV.write(local_f_path, df_sub, delim='\t')
    run(`chmod 777 $(local_f_path)`)
    f_url = "file:///$(local_f_path)"
    cmd =
    """
    LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
    CREATE (node:$(NODE_TYPE) $(parameters))
    """
    cmd = rstrip(replace(cmd, '\n' => ' '))
    cypher_cmd = Mycelia.cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd)
    run(cypher_cmd) 
end

Progress: 100%|█████████████████████████████████████████| Time: 0:09:09


In [86]:
# add the edges to it

In [87]:
src_type = dst_type = "Taxonomy"
edge_type = "IS_PARENT_OF"
# upload_edge_type_over_url(src_type, dst_type, edge_type, graph, ADDRESS, USERNAME, PASSWORD, DATABASE)

"IS_PARENT_OF"

In [88]:
window_size = 10000
V = DataFrames.nrow(edge_table)
windows = [i:min(i+window_size-1,V) for i in 1:window_size:V]

243-element Vector{UnitRange{Int64}}:
 1:10000
 10001:20000
 20001:30000
 30001:40000
 40001:50000
 50001:60000
 60001:70000
 70001:80000
 80001:90000
 90001:100000
 100001:110000
 110001:120000
 120001:130000
 ⋮
 2310001:2320000
 2320001:2330000
 2330001:2340000
 2340001:2350000
 2350001:2360000
 2360001:2370000
 2370001:2380000
 2380001:2390000
 2390001:2400000
 2400001:2410000
 2410001:2420000
 2420001:2420860

In [90]:
ProgressMeter.@showprogress for (i, w) in enumerate(windows)
    df_sub = edge_table[w, :]
    f = "edge$i.tsv"
    local_f_path = "$(temp_upload_dir)/$(f)"
    uCSV.write(local_f_path, df_sub, delim='\t')
    run(`chmod 777 $(local_f_path)`)
    f_url = "file:///$(local_f_path)"
    cmd = 
    """
    LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
    MATCH (src:$(src_type) {identifier: row.src})
    MATCH (dst:$(dst_type) {identifier: row.dst})
    MERGE (src)-[p:$(edge_type)]->(dst)
    """
    cmd = rstrip(replace(cmd, '\n' => ' '))
    cypher_cmd = Mycelia.cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd)
    run(cypher_cmd) 
end

Progress: 100%|█████████████████████████████████████████| Time: 0:06:18


In [91]:
# ADDRESS
# a = "neo4j+s://41a45c7d.databases.neo4j.io"

In [94]:
a = replace(ADDRESS, ":7687" => "")

"neo4j+s://41a45c7d.databases.neo4j.io"

In [97]:
# run(`sudo touch /etc/neo4j/neo4j.conf`)
run(`sudo neo4j stop`)

Stopping Neo4j....... stopped.


Process(`sudo neo4j stop`, ProcessExited(0))

In [100]:
# remote database needs to be running
# needs to be big enough
# leave off port from address

# run(`neo4j-admin push-to-cloud --overwrite --verbose --bolt-uri=$(ADDRESS) --username=$(USERNAME) --password=$(PASSWORD)`)
# run(`sudo neo4j-admin push-to-cloud --overwrite --verbose --dump-to "$(DIR)/test.db.dump" --bolt-uri=$(a) --username=$(USERNAME) --password=$(PASSWORD)`)
run(`sudo neo4j-admin push-to-cloud --overwrite --verbose --bolt-uri=$(a) --username=$(USERNAME) --password=$(PASSWORD)`)

Selecting JVM - Version:11.0.15+10-Ubuntu-0ubuntu0.18.04.1, Name:OpenJDK 64-Bit Server VM, Vendor:Private Build


Executing command line: /usr/lib/jvm/java-11-openjdk-amd64/bin/java -cp /etc/neo4j/*:/usr/share/neo4j/lib/commons-beanutils-1.9.4.jar:/usr/share/neo4j/lib/stax-ex-1.8.1.jar:/usr/share/neo4j/lib/jakarta.annotation-api-1.3.5.jar:/usr/share/neo4j/lib/neo4j-cypher-ast-factory-4.4.6.jar:/usr/share/neo4j/lib/shiro-core-1.8.0.jar:/usr/share/neo4j/lib/neo4j-procedure-4.4.6.jar:/usr/share/neo4j/lib/javax.ws.rs-api-2.1.1.jar:/usr/share/neo4j/lib/neo4j-resource-4.4.6.jar:/usr/share/neo4j/lib/FastInfoset-1.2.16.jar:/usr/share/neo4j/lib/jakarta.validation-api-2.0.2.jar:/usr/share/neo4j/lib/neo4j-data-collector-4.4.6.jar:/usr/share/neo4j/lib/jctools-core-3.3.0.jar:/usr/share/neo4j/lib/commons-logging-1.2.jar:/usr/share/neo4j/lib/javassist-3.25.0-GA.jar:/usr/share/neo4j/lib/neo4j-lucene-index-4.4.6.jar:/usr/share/neo4j/lib/eclipse-collections-api-10.4.0.jar:/usr/share/neo4j/lib/netty-transport-4.1.75.Final.jar:/usr/share/neo4j/lib/netty-buffer-4.1.75.Final.jar:/usr/share/neo4j/lib/neo4j-storage-engin


Files: 1/51, data:  0.0%
Files: 1/51, data:  0.0%
Files: 2/51, data:  0.0%
Files: 3/51, data:  0.0%
Files: 4/51, data:  0.6%
Files: 4/51, data:  1.6%
Files: 4/51, data:  2.0%
Files: 5/51, data:  2.0%
Files: 6/51, data:  2.7%
Files: 6/51, data:  3.9%
Files: 6/51, data:  4.1%
Files: 7/51, data:  4.1%
Files: 8/51, data:  5.2%
Files: 8/51, data:  6.0%
Files: 8/51, data:  6.1%
Files: 9/51, data:  7.2%
Files: 9/51, data:  8.3%
Files: 9/51, data:  9.6%
Files: 9/51, data: 10.9%
Files: 9/51, data: 12.2%
Files: 9/51, data: 13.5%
Files: 9/51, data: 14.5%
Files: 9/51, data: 15.7%
Files: 9/51, data: 16.9%
Files: 9/51, data: 18.2%
Files: 9/51, data: 19.5%
Files: 9/51, data: 20.6%
Files: 9/51, data: 22.0%
Files: 9/51, data: 23.1%
Files: 9/51, data: 24.1%
Files: 9/51, data: 25.1%
Files: 9/51, data: 25.7%
Files: 10/51, data: 25.7%
Files: 11/51, data: 25.7%
Files: 12/51, data: 25.7%
Files: 13/51, data: 25.7%
Files: 14/51, data: 25.7%
Files: 15/51, data: 25.7%
Files: 16/51, data: 25.7%
Files: 17/51, dat

Dumped contents of database 'neo4j' into '/var/lib/neo4j/dump-of-neo4j-1652640983971'
Determined DumpSize=4965334842 bytes from dump at /var/lib/neo4j/dump-of-neo4j-1652640983971
Validating sizes: fromDump=4965334842, fromDatabase=4249343070Warning: unexpectedly large difference between size in dump, and original size: 4965334842 != 4249343070Uploading data of 4.6 GB to https://console.neo4j.io/v1/databases/41a45c7d
Got json '{"SignedURI":"https://storage.googleapis.com/data-import-uploads-production/41a45c7d-3512359765.dump?GoogleAccessId=admin-worker@neo4j-cloud.iam.gserviceaccount.com&Expires=1655232998&Signature=taZ9UgCRrwTx9cDOLpxPgE1NMv9UzFLfHUOKcssjgPeOAfg1ju27lLFd7jjou5%2FK2v5O9t%2FY2Ap6Qz71Es%2BDoE1ZvI%2F2EFe23%2B322ZvSfA0YK0Gdxamdzd%2FwcLn6vSCP36s8gKbhorpvZuVTGUhm%2BhxG64Px0zI2mEdYyT43cP3Zl8s%2B3DeAhmDXZT0ULOsUm3UL8cdNTxFZ5JCDGG%2Bl2j%2BKKbsdEd3cEG83vQL%2BuQBnIByJQkl3oLj7FqYhwPyMvpC%2FA%2FjwI0FCtWB9FMBGuOMN0pWb75C5DZKhth7vMAMpxxnVvOq%2B5n8bRXkgHilPBz8PyzFn9G%2BG2JApZ18m0w%3D%

Process(`sudo neo4j-admin push-to-cloud --overwrite --verbose --bolt-uri=neo4j+s://41a45c7d.databases.neo4j.io --username=neo4j --password=PtvnGnnS58elLLVsGwfeovOLi5xdfPe0hTMrwa_WtbM`, ProcessExited(0))